Copyright (c) 2020. Cognitive Scale Inc. All rights reserved.
Licensed under CognitiveScale Example Code [License](https://github.com/CognitiveScale/cortex-certifai-examples/blob/7998b8a481fccd467463deb1fc46d19622079b0e/LICENSE.md)

# Introduction

This fifth notebook shows you how to explore the trust score results in a notebook. If you have not already done so, please run the [fourth notebook](patient-readmission-trust-scan.ipynb) to scan the models for their trust scores.

In this notebook, we will:
1. Load the previously saved trust score reports
2. Convert the results into a dataframe and display them


In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

from certifai.scanner.report_reader import ScanReportReader
from certifai.scanner.report_utils import (
    scores, construct_scores_dataframe, construct_feature_scores_dataframe, construct_overall_scores_dataframe,
    construct_group_scores_dataframe)
from certifai.scanner.builder import ExplanationType
from IPython.display import display, Markdown

# Loading the Trust Score Reports

To load the reports, we need to know the use case ID ('readmission') and the scan ID.

List the available use cases, and the scans within the 'readmission' use case.

In [2]:
reader = ScanReportReader("reports")
reader.list_usecases()
scans = reader.list_scans('readmission')
data=[[s['date'], ', '.join(s['reportTypes']), s['id']] for s in scans]
df = pd.DataFrame(data, columns=['date', 'evals', 'scan id']).sort_values(by=['date'], ascending=False)
print(df)

              date                                      evals       scan id
0  20201204T092603  explainability, fairness, atx, robustness  9a4c16e5931f
1  20201204T084020                                explanation  aee134c9b4e9


Locate the latest trust score scan and load it.

In [3]:
latest_scan = df[df['evals'].str.contains('fairness') & df['evals'].str.contains('robustness')].iloc[0]
results = reader.load_scan('readmission', latest_scan['scan id'])

# Extract the results

In this section we'll construct dataframes containing the score results and display them.

Print out the overall fairness results.

In [4]:
fairness_scores = list(scores('fairness', results))
display(construct_overall_scores_dataframe(fairness_scores, include_confidence=True))

,context,eval type,score type,score,score lower bound,score upper bound
0,logit,fairness,burden,56.780386,50.018976,62.938135
1,mlp,fairness,burden,64.347055,56.078975,71.632597


Print out the fairness scores by feature

In [5]:
display(construct_feature_scores_dataframe(fairness_scores, include_confidence=True))

,context,eval type,score type,age,age lower bound,age upper bound,gender,gender lower bound,gender upper bound,race,race lower bound,race upper bound
0,logit,fairness,burden,59.378397,53.162254,65.556422,78.21444,65.787082,90.454043,58.426784,50.321139,66.360269
1,mlp,fairness,burden,74.248612,69.106686,79.326453,82.86353,74.469198,92.070642,64.347055,56.078975,71.771835


Print out the detailed fairness scores

In [6]:
display(construct_group_scores_dataframe(fairness_scores, include_confidence=True))

,context,eval type,score type,value type,age - 15,age - 15 lower bound,age - 15 upper bound,age - 25,age - 25 lower bound,age - 25 upper bound,...,race - Caucasian upper bound,race - Hispanic,race - Hispanic lower bound,race - Hispanic upper bound,race - Other,race - Other lower bound,race - Other upper bound,race - nan,race - nan lower bound,race - nan upper bound
0,logit,fairness,burden,burden,0.042295,0.028121,0.058176,0.048553,0.021672,0.084202,...,0.044633,0.039058,0.025239,0.058705,0.017581,0.008425,0.029549,0.002436,0.000000,0.005221
1,mlp,fairness,burden,burden,0.080654,0.059399,0.105131,0.052011,0.038256,0.068210,...,0.054095,0.072680,0.050700,0.103020,0.032921,0.021367,0.046239,0.007532,0.003083,0.013656


For an example of visualizing the fairness results in a chart, see the [fairness metrics notebook](https://github.com/CognitiveScale/cortex-certifai-examples/blob/master/notebooks/fairness_metrics/FairnessMetrics.ipynb).

Print out the robustness scores.

In [7]:
df = construct_scores_dataframe(scores('robustness', results), include_confidence=True)
display(df)

,context,robustness,robustness lower bound,robustness upper bound
mlp,mlp,66.866121,61.640160,71.767569
logit,logit,80.519300,76.261203,84.208782


Print out the explainability scores.

In [8]:
df = construct_scores_dataframe(scores('explainability', results), include_confidence=False)
display(df)

,context,explainability,Num features (1),Num features (10),Num features (2),Num features (3),Num features (4),Num features (5),Num features (6),Num features (7),Num features (8),Num features (9)
mlp,mlp,96.796875,70.3125,0.0,28.1250,1.56250,0.0,0.0,0.00000,0.0,0.0,0.0
logit,logit,96.601562,76.5625,0.0,20.3125,2.34375,0.0,0.0,0.78125,0.0,0.0,0.0
